-create pipeline to scrape stories off of the SCP wiki
-run analysis on these stories, lots of different metrics (word count, find some python libraries to analyize the stories)
-plan on using some sort of machine learning algorithmn to find out what stories are closest to each other
-this way, can create something that recommends certain scps based on other ones


-NLTK (natural language tool kit)
-get sentiment

### Ideas for parts of dataframe:
- scp number
- rating
- D-class involved? (boolean, if the word D-class found in entry)
- Object Class
- tags
- word count
- sub scps
- sentiment

In [1]:
# random scp generator
import random

def rand_scp_num(x = 1):
    ''' generates x number of random numbers from 1-5999 that corresponds to an scp page
    Args:
        x = 1 (int): integer for amount of scp numbers you want returned
    Returns:
        scp_num (list): a list with values from 1-5999, formatted to correctly open an scp page
    '''
    # create list to store scp numvers in
    scp_nums = []
    for i in range(x):
        n = "000" + str(random.randint(1, 5999))
        while len(n) > 4:
            n = n.replace("0", "", 1)
        if n[0] == "0":
            n = n.replace("0", "", 1)
        scp_nums.append(n)
    return(scp_nums)

In [2]:
# function to input scp number into and return soup object for that scp's webpage
import requests
import json
from bs4 import BeautifulSoup

def get_scp_soup(scp):
    '''gets Beautiful Soup of the given scp number
    Args:
        scp (int): the number of the scp 
    Returns:
        scp_soup (Beautiful Soup object): soup object from the scp html
    '''
    # first, it gets the html for the given scp
    url = f'https://scp-wiki.wikidot.com/scp-{scp}'
    html = requests.get(url).text
    
    # get the soup for the html
    scp_soup = BeautifulSoup(html)
    
    # return the soup
    return(scp_soup)

In [3]:
def get_scp_maintext(num, scp_soup):
    '''gets the main story text from the scp soup object
    Args:
        num (int): the number of the scp the soup is for
        scp_soup (Beautiful Soup object): soup object for the scp's webpage
    Returns:
        scp_story (str): string that contains the story from the scp webpage
    '''
    # get the text using div and page-content from the soup
    scp_fulltext = (scp_soup.find("div", id = "page-content")).text
    
    # the text potentially has things before and after the actual story, so subset this text into just the story
    # we are assuming the story starts once the item is named (Item #: SCP-XXXX)
    # we are assuming the story ends when the bottom of the screen displays links to the next/previous story (« SCP-XXXX - 1)
    
    # create the words that indicate the story starts/stops and find its location in the full text
    start = scp_fulltext.find(f'Item #: SCP-{num}')
    stop = scp_fulltext.find(f'« SCP-{int(num)-1}')
    
    # subset the full text by these location
    scp_story = scp_fulltext[start : stop]
    
    return(scp_story)
    

In [4]:
# gets positive and overall rating for the scp
def get_rating(soup):
    '''gets the positive and overall ratings for the scp
    Args:
        soup (Beautiful Soup object): soup for the scp
    Return:
        ratings (list): list with positive rating, overall rating    
    '''
    # get the 8th class=image, which is the image of the rating box, and get the src, which is the link
    text = soup.find_all(class_ = "image")[7]["src"]
    
    # clean the link to get the ratings from it
    link = text.split("&")
    ratings = [int(link[3].replace("rating=", "")), int(link[4].replace("rating_votes=", ""))]
    
    return(ratings)

In [12]:
import string
def get_subobjects(num, text):
    '''gets a list of the letters of all subobjects for an scp
    Args:
        num (int): the number of the scp
        text (str): the text of the main scp story
    Return:
        sub_objects (list): a list of the letters for all sub objects    
    '''
    
    '''
    # create list to add subobjects to
    sub_objects = []
    
    # create letter of sub object to test for
    let = "0"

    # boolean to test in for loop- set to true to start loop
    is_sub = True

    # while the boolean is true (while the sub object exists) continue to loop
    while is_sub == True:
        # see if subobject exists
        is_sub = f'SCP-{num}-{let}' in text
        
        # if it exsits, add the letter into list of subobjects and then make the letter the next letter
        if is_sub == True:
            sub_objects.append(let)
            let = chr(ord(let) + 1)
    
    return(sub_objects)
    
    '''
    
    sub_objects = []
    
    lets = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    for j in list(string.ascii_uppercase):
        lets.append(j)
    
    for i in lets:
        is_sub = f'SCP-{num}-{i}' in text
        # if it exsits, add the letter into list of subobjects and then make the letter the next letter
        if is_sub == True:
            sub_objects.append(i)
            
    if len(sub_objects) > 0:
        return sub_objects
    else:
        return None

In [6]:
def get_tags(soup):
    ''' gets the tags on the bottom of the page for the scp
    Args:
        soup (Beautiful Soup object): the beautiful soup object for the scp
    Returns:
        tags (list): a list of the tags
    '''
    # get the part of the soup that has the page tags
    scp_tags = soup.find(class_ = "page-tags")
    
    # the page tags is an amalgamation of things, seperate them by their a to get a list of each tag
    scp_tags_list = scp_tags.find_all("a")

    # loop over the tags and add the text of all the tags to a list
    tags = []
    for tag in scp_tags_list:
        # some tags start with"_", we dont want those, so only add ones that don't start with "_"
        if (tag.text)[0] != "_":
            tags.append(tag.text)
        
    return(tags)

In [7]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

# creates the dataframe!!!
def scp_dataframe(x=1):
    '''creates dataframe with attributes for a number of random scps
    Args:
        x=1 (int): optional number of how many scps to put in the dataframe
    Return:
        df_scp (df): dataframe with attributes for a number of random scps
        '''    
    # create dataframe with column names but no data
    df_scp = pd.DataFrame(columns = ["Number", "Class", "Pos Ratings", "Pos Rating Rate", "Sub-Objects", "D-Class", "Tags", "Text", "Word Count", "Sentiment"])
    
    # create x random scp numbers
    scp_nums = rand_scp_num(x)
    
    # loops through each scp to gather and add attributes for each scp
    for num in scp_nums:
        # create dic to add scp attributes to, to eventually add to dataframe
        dic_scp = {}

        # get the random scp's soup
        scp_soup = get_scp_soup(num)
        
        # get rating of scp
        ratings = get_rating(scp_soup)
        
        # get the tags of the scp
        tags = get_tags(scp_soup)
        
        # get the main text of the scp
        text = get_scp_maintext(num, scp_soup)
        if text == '':
            print("Page contains adult content. Adding different SCP to list.")
            scp_nums.append(rand_scp_num(x=1)[0])
            continue
            
        scp_class = get_class(text)
        
        word_count = len(text.split())
        
        # get list of sub objects
        sub_objects = get_subobjects(num, text)
        
        # get boolean of if D-Class is mentioned in text
        dclass = "D-Class" in text
        
        
        # create dic to add scp attributes to, to eventually add to dataframe
        dic_scp = {"Number" : num, "Class" : scp_class, "Pos Ratings" : ratings[0], "Pos Rating Rate" : ratings[0]/ratings[1], 
                   "D-Class" : dclass, "Tags" : tags, "Text" : text, "Sub-Objects" : sub_objects, "Word Count" : word_count, "Sentiment" : None}
        
        # add dict as another row in the scp dataframe
        df_scp = df_scp.append(dic_scp, ignore_index=True)
    df_scp = df_scp.set_index("Number")
    
    return(df_scp)

In [8]:
def get_class(text):
    '''returns class of SCP based on SCP text
    Args:
        text (str): text of SCP page (according to scp_dataframe())
    Return:
        class (str): string of SCP class (e.g. 'Euclid')
    '''
    try:
        start = text.index("Object Class:")
    except ValueError:
        return None
    text = text[start+14:]
    end = text.index("\n")
    
    return text[:end]

In [9]:
# testing
# get multiple scps to test on
scp_4006 = get_scp_soup(4006)
scp_3698 = get_scp_soup(3698)
scp_1530 = get_scp_soup(1530)
scp_400 = get_scp_soup(400) # has 2 subclasses

In [14]:
df_scp = scp_dataframe(3)
df_scp

,Class,Pos Ratings,Pos Rating Rate,Sub-Objects,D-Class,Tags,Text,Word Count,Sentiment
Number,,,,,,,,,
2379,Keter,100,0.746269,None,False,"[addictive, compulsion, fungus, keter, meteoro...",Item #: SCP-2379\nObject Class: Keter\nSpecial...,722,None
747,Euclid,241,0.876364,"[0, B]",False,"[euclid, humanoid, kindness, predatory, scp, s...",Item #: SCP-747\nObject Class: Euclid\nSpecial...,1108,None
4323,Keter,36,0.642857,None,False,"[biohazard, invertebrate, keter, scp, self-rep...",Item #: SCP-4323\nObject Class: Keter\nSpecial...,429,None


In [11]:
pass